### Libraries

In [54]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score


### Data Loading Reading

In [55]:
fake=pd.read_csv('Fake.csv')
true=pd.read_csv('True.csv')

In [56]:
fake.head()

title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  \
0  Donald Trump just couldn t wish all Americans ...    News   
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

                date  
0  December 31, 2017  
1  December 31, 2017  
2  December 30, 2017  
3  December 29, 2017  
4  December 25, 2017

In [57]:
true.head()

title  \
0  As U.S. budget fight looms, Republicans flip t...   
1  U.S. military to accept transgender recruits o...   
2  Senior U.S. Republican senator: 'Let Mr. Muell...   
3  FBI Russia probe helped by Australian diplomat...   
4  Trump wants Postal Service to charge 'much mor...   

                                                text       subject  \
0  WASHINGTON (Reuters) - The head of a conservat...  politicsNews   
1  WASHINGTON (Reuters) - Transgender people will...  politicsNews   
2  WASHINGTON (Reuters) - The special counsel inv...  politicsNews   
3  WASHINGTON (Reuters) - Trump campaign adviser ...  politicsNews   
4  SEATTLE/WASHINGTON (Reuters) - President Donal...  politicsNews   

                 date  
0  December 31, 2017   
1  December 29, 2017   
2  December 31, 2017   
3  December 30, 2017   
4  December 29, 2017

### Data Exploration

In [58]:
fake['label']=0

In [59]:
true['label']=1

In [60]:
true.head()

title  \
0  As U.S. budget fight looms, Republicans flip t...   
1  U.S. military to accept transgender recruits o...   
2  Senior U.S. Republican senator: 'Let Mr. Muell...   
3  FBI Russia probe helped by Australian diplomat...   
4  Trump wants Postal Service to charge 'much mor...   

                                                text       subject  \
0  WASHINGTON (Reuters) - The head of a conservat...  politicsNews   
1  WASHINGTON (Reuters) - Transgender people will...  politicsNews   
2  WASHINGTON (Reuters) - The special counsel inv...  politicsNews   
3  WASHINGTON (Reuters) - Trump campaign adviser ...  politicsNews   
4  SEATTLE/WASHINGTON (Reuters) - President Donal...  politicsNews   

                 date  label  
0  December 31, 2017       1  
1  December 29, 2017       1  
2  December 31, 2017       1  
3  December 30, 2017       1  
4  December 29, 2017       1

In [61]:
fake.head()

title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  \
0  Donald Trump just couldn t wish all Americans ...    News   
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

                date  label  
0  December 31, 2017      0  
1  December 31, 2017      0  
2  December 30, 2017      0  
3  December 29, 2017      0  
4  December 25, 2017      0

In [62]:
fake.count()

title      23481
text       23481
subject    23481
date       23481
label      23481
dtype: int64

In [63]:
fake.isnull().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [64]:
true.isnull().sum

<bound method DataFrame.sum of        title   text  subject   date  label
0      False  False    False  False  False
1      False  False    False  False  False
2      False  False    False  False  False
3      False  False    False  False  False
4      False  False    False  False  False
...      ...    ...      ...    ...    ...
21412  False  False    False  False  False
21413  False  False    False  False  False
21414  False  False    False  False  False
21415  False  False    False  False  False
21416  False  False    False  False  False

[21417 rows x 5 columns]>

In [65]:
true.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
 4   label    21417 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 836.7+ KB


In [66]:
fake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
 4   label    23481 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 917.4+ KB


In [67]:
for i in true.columns:
    print(i)

title
text
subject
date
label


In [68]:
true['subject'].unique()

array(['politicsNews', 'worldnews'], dtype=object)

In [69]:
fake['title'].unique()

array([' Donald Trump Sends Out Embarrassing New Year’s Eve Message; This is Disturbing',
       ' Drunk Bragging Trump Staffer Started Russian Collusion Investigation',
       ' Sheriff David Clarke Becomes An Internet Joke For Threatening To Poke People ‘In The Eye’',
       ...,
       'BOILER ROOM – No Surrender, No Retreat, Heads Will Roll  – EP #38',
       'Federal Showdown Looms in Oregon After BLM Abuse of Local Ranching Family – Bundys Lead Protest',
       'A Troubled King: Chicago’s Rahm Emanuel Desperate to Save His 2020 Presidential Run'],
      dtype=object)

In [70]:
true.nunique()

title      20826
text       21192
subject        2
date         716
label          1
dtype: int64

In [71]:
fake.nunique

<bound method DataFrame.nunique of                                                    title  \
0       Donald Trump Sends Out Embarrassing New Year’...   
1       Drunk Bragging Trump Staffer Started Russian ...   
2       Sheriff David Clarke Becomes An Internet Joke...   
3       Trump Is So Obsessed He Even Has Obama’s Name...   
4       Pope Francis Just Called Out Donald Trump Dur...   
...                                                  ...   
23476  McPain: John McCain Furious That Iran Treated ...   
23477  JUSTICE? Yahoo Settles E-mail Privacy Class-ac...   
23478  Sunnistan: US and Allied ‘Safe Zone’ Plan to T...   
23479  How to Blow $700 Million: Al Jazeera America F...   
23480  10 U.S. Navy Sailors Held by Iranian Military ...   

                                                    text      subject  \
0      Donald Trump just couldn t wish all Americans ...         News   
1      House Intelligence Committee Chairman Devin Nu...         News   
2      On Friday, it was 

In [72]:
news=pd.concat([fake,true],axis=0)

In [73]:
news.head()

title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  \
0  Donald Trump just couldn t wish all Americans ...    News   
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

                date  label  
0  December 31, 2017      0  
1  December 31, 2017      0  
2  December 30, 2017      0  
3  December 29, 2017      0  
4  December 25, 2017      0

In [74]:
news.tail()

title  \
21412  'Fully committed' NATO backs new U.S. approach...   
21413  LexisNexis withdrew two products from Chinese ...   
21414  Minsk cultural hub becomes haven from authorities   
21415  Vatican upbeat on possibility of Pope Francis ...   
21416  Indonesia to buy $1.14 billion worth of Russia...   

                                                    text    subject  \
21412  BRUSSELS (Reuters) - NATO allies on Tuesday we...  worldnews   
21413  LONDON (Reuters) - LexisNexis, a provider of l...  worldnews   
21414  MINSK (Reuters) - In the shadow of disused Sov...  worldnews   
21415  MOSCOW (Reuters) - Vatican Secretary of State ...  worldnews   
21416  JAKARTA (Reuters) - Indonesia will buy 11 Sukh...  worldnews   

                   date  label  
21412  August 22, 2017       1  
21413  August 22, 2017       1  
21414  August 22, 2017       1  
21415  August 22, 2017       1  
21416  August 22, 2017       1

In [75]:
news.isnull().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [76]:
news=news.drop(['title','subject','date'],axis=1)

In [77]:
news.head()

text  label
0  Donald Trump just couldn t wish all Americans ...      0
1  House Intelligence Committee Chairman Devin Nu...      0
2  On Friday, it was revealed that former Milwauk...      0
3  On Christmas day, Donald Trump announced that ...      0
4  Pope Francis used his annual Christmas Day mes...      0

In [78]:
news=news.sample(frac=1)

In [79]:
news.head()

text  label
9759   Late-night TV host Jimmy Kimmel opened his pro...      0
2622   WASHINGTON (Reuters) - The U.S. Senate Judicia...      1
11897  Lady Liberty will be depicted as a black women...      0
8212   When you ve never been taught what a figure of...      0
14333  Do you have a friend, a co-worker or (in my ca...      0

In [80]:
news.reset_index(inplace=True)

In [81]:
news.head()

index                                               text  label
0   9759  Late-night TV host Jimmy Kimmel opened his pro...      0
1   2622  WASHINGTON (Reuters) - The U.S. Senate Judicia...      1
2  11897  Lady Liberty will be depicted as a black women...      0
3   8212  When you ve never been taught what a figure of...      0
4  14333  Do you have a friend, a co-worker or (in my ca...      0

In [82]:
news.drop(['index'],axis=1,inplace=True)

In [83]:
news.head()

text  label
0  Late-night TV host Jimmy Kimmel opened his pro...      0
1  WASHINGTON (Reuters) - The U.S. Senate Judicia...      1
2  Lady Liberty will be depicted as a black women...      0
3  When you ve never been taught what a figure of...      0
4  Do you have a friend, a co-worker or (in my ca...      0

In [84]:
def wordopt(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^a-zA-Z]', ' ', text)  # Remove non-alphabetical characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text


In [85]:
news['text'] = news['text'].apply(wordopt)


In [86]:
news['text']

0        late night tv host jimmy kimmel opened his pro...
1        washington reuters the u s senate judiciary co...
2        lady liberty will be depicted as a black women...
3        when you ve never been taught what a figure of...
4        do you have a friend a co worker or in my case...
                               ...                        
44893    taipei reuters taiwan hopes to find a natural ...
44894    diamondandsilk wasn t pleased by the underhand...
44895    toronto reuters canada will expel a venezuelan...
44896    if you ever find yourself struggling to explai...
44897    it isn t much of a secret these days that rich...
Name: text, Length: 44898, dtype: object

In [87]:
x=news['text']
y= news['label']

In [88]:
x

0        late night tv host jimmy kimmel opened his pro...
1        washington reuters the u s senate judiciary co...
2        lady liberty will be depicted as a black women...
3        when you ve never been taught what a figure of...
4        do you have a friend a co worker or in my case...
                               ...                        
44893    taipei reuters taiwan hopes to find a natural ...
44894    diamondandsilk wasn t pleased by the underhand...
44895    toronto reuters canada will expel a venezuelan...
44896    if you ever find yourself struggling to explai...
44897    it isn t much of a secret these days that rich...
Name: text, Length: 44898, dtype: object

In [89]:
y

0        0
1        1
2        0
3        0
4        0
        ..
44893    1
44894    0
44895    1
44896    0
44897    0
Name: label, Length: 44898, dtype: int64

### Train-Test Splitting

In [90]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3)

In [91]:
x_test.shape

(13470,)

In [92]:
x_train.shape

(31428,)

In [93]:
y_test.shape

(13470,)

### Training Model Using Support Vector Classifier

In [94]:
vectorization = TfidfVectorizer(max_features=500)
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)


In [95]:
xv_train

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2965891 stored elements and shape (31428, 500)>

In [96]:
xv_test

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1263622 stored elements and shape (13470, 500)>

In [97]:
xv_train

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2965891 stored elements and shape (31428, 500)>

In [98]:

model = SVC(kernel='linear', random_state=42)  # Choose kernel based on requirements
model.fit(xv_train, y_train)


SVC(kernel='linear', random_state=42)

In [99]:
y_pred = model.predict(xv_test)


In [100]:

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))


Accuracy: 0.9906458797327394
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      7107
           1       0.99      0.99      0.99      6363

    accuracy                           0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470



In [101]:
from sklearn.metrics import accuracy_score
testdata_predict = model.predict(xv_test)
model_accuracy_score = accuracy_score(y_test, testdata_predict)
print("-- Model Accuracy Score: ", end='')
print(round(model_accuracy_score, 3))


-- Model Accuracy Score: 0.991
